In [3]:
import datetime
import warnings
import mysql.connector as msc
import yfinance as yf
import numpy
from pandas import Timestamp
warnings.filterwarnings('ignore')
# Obtain a database connection to the MySQL instance

db_host = 'localhost'
db_user = 'sec_user'
db_pass = 'Damilare20$'
db_name = 'securities_master'
plug ='caching_sha2_password'
con = msc.connect(host=db_host, user=db_user, password=db_pass, db=db_name, auth_plugin= plug)

In [4]:
def obtain_list_of_db_tickers():
    """
    Obtains a list of the ticker symbols in the database.
    """
    con = msc.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name, connect_timeout=28800)
    with con:
        cur = con.cursor()
        cur.execute("""SELECT securities_master.symbol.id, securities_master.symbol.ticker
        FROM securities_master.symbol """)
        data = cur.fetchall()
        return data

def convert_numpy_int_to_int(obj):
    for x in range(len(obj)):
        for y in range(len(obj[0])):
            if isinstance(obj[x][y], numpy.int64):
                obj[x] = list(obj[x])
                obj[x][y] = int(obj[x][y])
                obj[x] = tuple(obj[x])
            else:
                pass
    return obj


def get_daily_historic_data_yahoo(ticker, start_date=datetime.datetime(2001, 1, 1), end_date=datetime.date.today()):
    """
    Obtains data from Yahoo Finance returns and a list of tuples.
    ticker: Yahoo Finance ticker symbol, e.g. "GOOG" for Google, Inc.
    start_date: Start date in (YYYY, M, D) format
    end_date: End date in (YYYY, M, D) format
    """
    # Create ticker object
    ticker = yf.Ticker(ticker)
    h_df = ticker.history(start=start_date, end=end_date, interval='1d')
    h_df.reset_index(inplace = True)
    prices = []
    for p in range(h_df.shape[0]):
        tup = (h_df.iloc[p, 0], h_df.iloc[p, 1], h_df.iloc[p, 2], h_df.iloc[p, 3], h_df.iloc[p, 4], h_df.iloc[p, 5],
               h_df.iloc[p, 6], h_df.iloc[p, 7])
        prices.append(tup)
    return prices


def convert_to_daily_data(data_vendor_id, symbol_id, data):
    now = datetime.datetime.now()
    records = []
    for tup in data:
        new_tup = (data_vendor_id, symbol_id, tup[0], now, now, tup[1], tup[2], tup[3], tup[4], tup[5], tup[6])
        records.append(new_tup)
    return records


def update_the_pricing_data(data_vendor_id, symbol_id, data):
    now = datetime.datetime.now()
    records = []
    for tup in data:
        if tup(0) >= Timestamp('2024-07-10 00:00:00-0400', tz='America/New_York'):
            new = (data_vendor_id, symbol_id, tup[0], now, now, tup[1], tup[2], tup[3], tup[4], tup[5], tup[6])
            records.append(new)
    return records

def insert_daily_data_into_db(daily_data):
    """
    Takes a list of tuples of daily data and adds it to the
    MySQL database. Appends the vendor ID and symbol ID to the data.
    daily_data: List of tuples of the OHLC data (with
    adj_close and volume)
    """
    # Create the insert strings
    insert_str = """ INSERT INTO securities_master.daily_price
    (securities_master.daily_price.data_vendor_id, securities_master.daily_price.symbol_id,
    securities_master.daily_price.price_date, securities_master.daily_price.created_date,
    securities_master.daily_price.last_updated_date, securities_master.daily_price.open_price,
    securities_master.daily_price.high_price, securities_master.daily_price.low_price, 
    securities_master.daily_price.close_price, securities_master.daily_price.volume, 
    securities_master.daily_price.split) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    # Using the MySQL connection, carry out an INSERT INTO for every symbol
    con = msc.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name, connect_timeout=28800)
    with con:
        cur = con.cursor()
        cur.executemany(insert_str, daily_data)
        con.commit()


In [17]:
# data =  get_daily_historic_data_yahoo('SPY', start_date=datetime.datetime(2001, 1, 1), end_date=datetime.date.today())
# dyf_data = convert_numpy_int_to_int(data)
# new_data = convert_to_daily_data('1',503, dyf_data)
# insert_daily_data_into_db(new_data)
# print('successful')

successful


In [13]:
# if __name__ == "__main__":
#     # This ignores the warnings regarding Data Truncation
#     # from the Yahoo precision to Decimal(19,4) datatypes
#     warnings.filterwarnings('ignore')
#     # Loop over the tickers and insert the daily historical
#     # data into the database
# tickers = obtain_list_of_db_tickers()
# print(tickers)
#     len_tickers = len(tickers)
#     for i, t in enumerate(tickers):
#         print("Adding data for %s: %s out of %s" % (t[1], i+1, len_tickers))
#         yf_data = get_daily_historic_data_yahoo(t[1])
#         dyf_data = convert_numpy_int_to_int(yf_data)
#         new_data = convert_to_daily_data('1', t[0], dyf_data)
#         # new_data = update_the_pricing_data('1', t[0], dyf_data)
#         insert_daily_data_into_db(new_data)
#     print("Successfully added Yahoo Finance pricing data to DB.")

[(1, 'AOS'), (2, 'ABT'), (3, 'ABBV'), (4, 'ACN'), (5, 'ADBE'), (6, 'AMD'), (7, 'AES'), (8, 'AFL'), (9, 'A'), (10, 'APD'), (11, 'ABNB'), (12, 'AKAM'), (13, 'ALB'), (14, 'ARE'), (15, 'ALGN'), (16, 'ALLE'), (17, 'LNT'), (18, 'ALL'), (19, 'GOOGL'), (20, 'GOOG'), (21, 'MO'), (22, 'AMZN'), (23, 'AMCR'), (24, 'AMTM'), (25, 'AEE'), (26, 'AEP'), (27, 'AXP'), (28, 'AIG'), (29, 'AMT'), (30, 'AWK'), (31, 'AMP'), (32, 'AME'), (33, 'AMGN'), (34, 'APH'), (35, 'ADI'), (36, 'ANSS'), (37, 'AON'), (38, 'APA'), (39, 'AAPL'), (40, 'AMAT'), (41, 'APTV'), (42, 'ACGL'), (43, 'ADM'), (44, 'ANET'), (45, 'AJG'), (46, 'AIZ'), (47, 'T'), (48, 'ATO'), (49, 'ADSK'), (50, 'ADP'), (51, 'AZO'), (52, 'AVB'), (53, 'AVY'), (54, 'AXON'), (55, 'BKR'), (56, 'BALL'), (57, 'BAC'), (58, 'BAX'), (59, 'BDX'), (60, 'BRK.B'), (61, 'BBY'), (62, 'TECH'), (63, 'BIIB'), (64, 'BLK'), (65, 'BX'), (66, 'BK'), (67, 'BA'), (68, 'BKNG'), (69, 'BWA'), (70, 'BSX'), (71, 'BMY'), (72, 'AVGO'), (73, 'BR'), (74, 'BRO'), (75, 'BF.B'), (76, 'BLDR'),